In [ ]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import openpyxl

In [ ]:
param_dic = {
    "host"      : "127.0.0.1",
    "database"  : "taxis",
    "user"      : "postgres",
    "password"  : "Snowdav3"
}

In [ ]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print('Connection succesful')
    
    return conn

conn = connect(param_dic)
cursor = conn. cursor()

In [ ]:
def time_lapse(seconds):
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60
    
    days = seconds // seconds_in_day
    hours = (seconds - (days * seconds_in_day)) // seconds_in_hour
    minutes = (seconds - (days * seconds_in_day) - (hours * seconds_in_hour)) // seconds_in_minute
    
    return f"{days} days, {hours} hours, {minutes} minutes"

# Parte 1 - SQL

1. 
    a) ¿Puedes describirnos el procedimiento que has seguido para cargar los tres ficheros?

- Mediante Requests solicito el html vía http del link proporcionado.
- Con BeautifoulSoup filtro por los div por año mediante el tag id="faq{year}". 
- De estos divs seleceiono los tag 'ul' (que contienen los links csv href) para los meses indicados

1.    
    b) ¿Puedes compartirnos la definición de la tabla que estás usando para contestar las
preguntas planteadas?

2. Una vez cargados los tres ficheros en tu base de datos, ¿cuántos registros tiene la
tabla? Compártenos la SQL que has usado.

In [ ]:
registros = '''SELECT COUNT(vendorid) AS registros_totales
FROM yellowtaxis;
'''
conn.rollback()
cursor.execute(registros)
result = cursor.fetchall()[0]
print(f"La tabla yellowtaxis de la database {conn.info.dbname} contiene {result[0]} registros")

3. Para cada uno de los tres meses con datos ¿podrías decirnos cuál es el trayecto más
corto, más largo y el trayecto medio?, en distancia y en tiempo. Compártenos la
sentencia SQL que has usado.


In [ ]:
def execute(month: int, trip_metric, trip_measure):
    
    query = None
    if trip_metric == 'longest':
        if trip_measure == 'time':
            longest_trip_time = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime, trip_duration_seconds, trip_duration
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_duration_seconds DESC
                LIMIT 1;
            '''
            query = longest_trip_time
        elif trip_measure == 'distance':
            longest_trip_distance = f'''
                SELECT vendorid, month, tpep_pickup_datetime, tpep_dropoff_datetime,
                trip_duration_seconds, trip_duration, trip_distance
                FROM yellowtaxis
                WHERE month = '{month}'
                ORDER BY trip_distance DESC
                LIMIT 1;
            '''
            query = longest_trip_distance
            
    elif trip_metric == 'shortest':
        if trip_measure == 'time':
            shortest_trip_time = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_duration_seconds ASC,
                trip_distance ASC
                LIMIT 1;

            '''
            query = shortest_trip_time
            
        elif trip_measure == 'distance':
            shortest_trip_distance = f'''
                SELECT *
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0
                ORDER BY trip_distance ASC,
                trip_duration_seconds ASC
                LIMIT 1;

            '''
            query = shortest_trip_distance
    
    elif trip_metric == 'average':
        if trip_measure == 'time':
            avg_trip_time = f'''
                SELECT AVG(trip_duration_seconds) AS trip_duration_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query = avg_trip_time
            
        elif trip_measure == 'distance':
            avg_trip_distance = f'''
                SELECT AVG(trip_distance) AS trip_distance_avg
                FROM yellowtaxis
                WHERE month = '{month}' AND trip_distance > 0 AND trip_duration_seconds > 0

            '''
            query =  avg_trip_distance
    
    cursor.execute(query)
    result = cursor.fetchall()[0]
    
    return result

# January

In [ ]:
# Longest time
jan_lg_t = execute(1, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from January 2020 took: {jan_lg_t[-1]}")
print()

# Longest distance

jan_lg_d = execute(1, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from January 2020 was: {jan_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
jan_st_t = execute(1, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from January 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
jan_st_d = execute(1, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from January 2020 was: {jan_st_d[7]} miles")
print()

In [ ]:
# Average
jan_avg_t =  execute(1, 'average', 'time')
print()
print(f"Average yellow taxi trip time from January 2020 was: {time_lapse(jan_avg_t[0])}")
print()

# Average distance
jan_avg_d = execute(1, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from January 2020 was: {jan_avg_d[0]} miles")
print()

# February 

In [ ]:
feb_lg_t = execute(2, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from February 2020 took: {feb_lg_t[-1]}")
print()

# Longest distance

feb_lg_d = execute(2, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from February 2020 was: {feb_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
feb_st_t = execute(2, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {feb_st_t[4]} seconds")
print()

# Shortest distance
feb_st_d = execute(2, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from February 2020 was: {feb_st_d[7]} miles")
print()

In [ ]:
# Average
feb_avg_t =  execute(2, 'average', 'time')
print()
print(f"Average yellow taxi trip time from February 2020 was: {time_lapse(feb_avg_t[0])}")
print()

# Average distance
feb_avg_d = execute(2, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from February 2020 was: {feb_avg_d[0]} miles")
print()

# March

In [ ]:
mar_lg_t = execute(3, 'longest', 'time')
print()
print(f"Longest yellow taxi trip time from March 2020 took: {mar_lg_t[-1]}")
print()

# Longest distance
mar_lg_d = execute(3, 'longest', 'distance')
print()
print(f"Longest yellow taxi trip distance from March 2020 was: {mar_lg_d[-1]} miles")
print()

In [ ]:
# Shortest time
mar_st_t = execute(3, 'shortest', 'time')
print()
print(f"Shortest yellow taxi trip time from February 2020 took: {jan_st_t[4]} seconds")
print()

# Shortest distance
mar_st_d = execute(3, 'shortest', 'distance')
print()
print(f"Shortest yellow taxi trip distance from March 2020 was: {mar_st_d[7]} miles")
print()

In [ ]:
# Average
mar_avg_t =  execute(3, 'average', 'time')
print()
print(f"Average yellow taxi trip time from March 2020 was: {time_lapse(mar_avg_t[0])}")
print()

# Average distance
mar_avg_d = execute(3, 'average', 'distance')
print()
print(f"Average yellow taxi trip distance from March 2020 was: {mar_avg_d[0]} miles")
print()

4. Para cada uno de los tres meses con datos ¿podrías decirnos cuánta es la variación
porcentual en el número de servicios registrados con respecto al mes anterior? Por
ejemplo, en esta pregunta nos gustaría obtener un resultado así. Compártenos la
sentencia SQL que has usado

# Parte 2 - Django

# Parte 3 - Reporting

- Empleando Python 3 generar un fichero Excel que contenga tres pestañas.
- Para este ejercicio las transformaciones queremos que se hagan en Pandas y no una
sentencia SQL
- Eplea el método/libreria de tu preferencia para generar el fichero Excel
- La información que queremos que se muestre en cada una de las pestañas es una
agrupación por mes y tipo de dia:
    - MES: Tendrá el formato año-mes, por ejemplo 2020-01
    -Tipo_dia: Tendrá el valor 1 si el viaje se realizó entre el lunes y el viernes
(ambos inclusive) y 2 si se hizo en fin de semana
    - Personas transportadas: El total de personas que han usado el servicio de Taxis
para ese mes y tipo_dia
    - Millas recorridas: El total de millas recorridas para ese mes y tipo_dia
    - Total servicios: El total de servicios que realizados para ese mes y tipo_dia

- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID
- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID
- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [ ]:
############################################################################################################

In [ ]:
name = "2020_yellow_jan_feb_mar.csv"
jan_feb_mar = pd.read_csv(name)

In [ ]:
jan_feb_mar

In [ ]:
month_col = 'month'
start_date_col = 'tpep_pickup_datetime'
final_date_col = 'tpep_dropoff_datetime'
passenger_col = 'passenger_count'
distance_col = 'trip_distance'
ratecode_col = 'RatecodeID'

df = jan_feb_mar.copy()

df = df[[month_col, start_date_col, final_date_col, passenger_col, distance_col, ratecode_col]]
df = df[(df[start_date_col] > '2020-01-01') & (df[start_date_col] <= '2020-03-31')]

type_object = df.select_dtypes(include='object').columns.to_list()
for elem in type_object:
        df[f"{elem}"] = pd.to_datetime(df[f"{elem}"], yearfirst=True, format="%Y/%m/%d %H:%M:%S")
        
df['day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
df['tipo_dia'] = df['day_of_week'].apply(lambda x: 1 if x in range(0,5) else 2)
df['total_servicios'] = 1
df['mes'] = df['tpep_pickup_datetime'].dt.strftime('%Y-%m')

df.rename(columns={"trip_distance": "millas_recorridas",
                   "passenger_count":"personas_transportadas"}, 
          inplace=True)


In [ ]:
df

In [ ]:
table = df.copy()
table = table[['mes', 'tipo_dia', 'personas_transportadas', 'millas_recorridas', 'total_servicios', 'RatecodeID']]
table

- Pestaña JFK: Tendrá la agrupación descrita para todos los viajes que tengan un 2 en la
variable RateCodeID

In [ ]:
# Agrupamos por las variables indicadas
jfk = table.copy()
jfk = jfk.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
jfk

In [ ]:
# Filtramos RaatecodeID = 2
jfk = jfk[jfk['RatecodeID'] == 2]
jfk

In [ ]:
jfk.drop(columns=['RatecodeID'], inplace=True)
jfk

In [ ]:
#table.groupby('mes').sum().reset_index()

- Pestaña regular: Tendrá la agrupación descrita para todos los viajes que tengan un 1 en
la variable RateCodeID

In [ ]:
regular = table.copy()
regular = regular.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
regular

In [ ]:
regular = regular[regular['RatecodeID'] == 1]
regular

In [ ]:
regular.drop(columns=['RatecodeID'], inplace=True)
regular

- Pestaña Others: Tendrá la agrupación descrita para todos los viajes que no tengan ni un
1 ni un 2 en la variable RateCodeID

In [ ]:
others = table.copy()
others = others.groupby(['mes', 'tipo_dia', 'RatecodeID']).sum().reset_index()
others

In [ ]:
others = others[(others['RatecodeID']) != 1 & (others['RatecodeID'] != 2)]
others.head(15)

In [ ]:
others.drop(columns=['RatecodeID'], inplace=True)
others

In [ ]:
with pd.ExcelWriter('report.xlsx') as writer:  
    jfk.to_excel(writer, sheet_name='JFK')
    regular.to_excel(writer, sheet_name='Regular')
    others.to_excel(writer, sheet_name='Others')

# Comprobaciones error de fuente

In [ ]:
raise ValueError("stop here")

Fichero CSV original de Enero contiene registros con fechas posteriores y anteriores

In [ ]:
jan = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2020-01.csv')
jan

In [ ]:
jan[(jan['tpep_pickup_datetime'] < '2020-01-01') | (jan['tpep_pickup_datetime'] > '2020-03-31')]